In [1]:
import pandas as pd
import numpy as np

In [2]:
double = pd.read_csv("Medication One Row Extraction/Two Combination One Row.csv")
single = pd.read_csv("Medication Extraction/Double Combination.csv")

In [3]:
double.head()

,Unnamed: 0,ID,Line,Medication,PFS,Duration
0,1,MM1,1,"Bortezomib, Lenalidomide",311,254
1,2,MM1,2,Stem cell transplant,1,1
2,3,MM10,1,"Bortezomib, Lenalidomide",91,91
3,4,MM10,2,Lenalidomide,62,1
4,5,MM10,3,Lenalidomide,112,76


In [4]:
single.head()

,Unnamed: 0,ID,Line,DaysFromDxStart,DaysFromDxStop,MedTx,Duration,PFS
0,1,MM1,1,4,258,Lenalidomide,254,311
1,2,MM1,1,4,258,Bortezomib,254,311
2,3,MM1,2,315,316,Stem cell transplant,1,1
3,4,MM10,1,21,112,Bortezomib,91,91
4,5,MM10,1,21,112,Lenalidomide,91,91


In [7]:
line1 = double[double["Line"] == 1]
doubles = line1["Medication"].unique()
doubles

array(['Bortezomib, Lenalidomide', 'Bortezomib, Thalidomide',
       'Bortezomib, Cyclophosphamide', 'Adriamycin, Vincristine',
       'Melphalan, Prednisone', 'Bortezomib, Doxorubicin',
       'Doxorubicin, Vincristine', 'Liposomal doxorubicin, Vincristine',
       'Cyclophosphamide, Etoposide', 'Bortezomib, Prednisone',
       'Carfilzomib, Cyclophosphamide', 'Carfilzomib, Lenalidomide',
       'Bortezomib, Liposomal doxorubicin', 'Elotuzumab, Lenalidomide',
       'Adriamycin, Bortezomib', 'Carfilzomib, Pomalidomide',
       'Cyclophosphamide, Paciltaxel', 'Bortezomib, Rituximab'],
      dtype=object)

In [18]:
def getSingle(med):
    doubleT = double[(double["Line"] != 1) & (double["Medication"] == med) & (double["ID"] == ID)]
    if doubleT.shape[0] != 0:
        return doubleT.iloc[0]["PFS"]
    else:
        stop = single[(single["Line"] == 1) & (single["MedTx"] == med) & (single["ID"] == ID)]["DaysFromDxStop"].values[0]
        start =  single[(single["Line"] == 1) & (single["MedTx"] == med) & (single["ID"] == ID)]["DaysFromDxStart"].values[0]
        return stop - start

In [19]:
for AB in doubles:
    AandB = AB.split(", ")
    A = AandB[0]
    B = AandB[1]
    clm = ["ID", "PFS_" + AB, "PFS_" + A, "PFS_" + B]
    ABpd = pd.DataFrame(columns = clm)
    medAB = line1[line1["Medication"] == AB]
    medAB = medAB[["ID", "PFS"]]
    IDs = medAB["ID"]
    count = 0
    for ID in IDs:
        PFS_A = getSingle(A)
        PFS_B = getSingle(B)
        PFS_AB = medAB[medAB["ID"] == ID]["PFS"].values[0]
        ABpd.loc[count] = [ID, PFS_AB, PFS_A, PFS_B]
        count = count + 1
    ABpd.to_csv("duo-regression/" + AB + ".csv")